In [150]:
# COLBERT MODEL
# -------------------------------------------------
# Fetch colBERT model basics and dependencies

!git -C ColBERT/ pull || git clone https://github.com/stanford-futuredata/ColBERT.git
import sys; sys.path.insert(0, 'ColBERT/')
!pip install torch==2.1.0+cu121 -f https://download.pytorch.org/whl/torch_stable.html


fatal: cannot change to 'ColBERT/': No such file or directory
Cloning into 'ColBERT'...
remote: Enumerating objects: 2610, done.
remote: Counting objects: 100% (1113/1113), done.
remote: Compressing objects: 100% (336/336), done.
remote: Total 2610 (delta 867), reused 832 (delta 777), pack-reused 1497
Receiving objects: 100% (2610/2610), 2.02 MiB | 5.59 MiB/s, done.
Resolving deltas: 100% (1626/1626), done.
Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [152]:
# Install additional dependencies
try:
    import google.colab
    !pip install -U pip
    !pip install -e ColBERT/['faiss-gpu']

except Exception:
  import sys; sys.path.insert(0, 'ColBERT/')
  try:
    from colbert import Indexer, Searcher
  except Exception:
    print("Check needed dependencies if you run outside Google Colab")
    assert False

Obtaining file:///content/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/ColBERT
  Preparing metadata (setup.py) ... done
  Attempting uninstall: colbert-ai
    Found existing installation: colbert-ai 0.2.18
    Uninstalling colbert-ai-0.2.18:
      Successfully uninstalled colbert-ai-0.2.18
  Running setup.py develop for colbert-ai


In [153]:
import colbert

In [154]:
from colbert import Indexer, Searcher
from colbert.infra import Run, RunConfig, ColBERTConfig
from colbert.data import Queries, Collection

In [155]:
# Import needed libraries and modules
from google.colab import drive
import os
import pandas as pd
import importlib

In [156]:
# Fetch docs from drive
drive.mount('/content/drive', force_remount = True)
docs_folder = '/content/drive/MyDrive/docs'

Mounted at /content/drive


In [162]:
# Create collection of documents
def create_collection(folder_path):
  collection = []

  # Iterate through each file in the folder
  for filename in os.listdir(docs_folder):
    file_path = os.path.join(docs_folder, filename)

    # Read lines from the file
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    text = ' '.join(lines).replace('\n', ' ')

    # Append the text of each doc to the list
    collection.append(text)
  return collection

collection = create_collection(docs_folder)



In [214]:
# Include processed tsv files from my GitHub Repo
!git clone https://github.com/AurumAsinus/IR_project.git
%cd IR_project
# File for processing of given theoretical relevance scores
import scores
# VSM Creation File
import Step2

fatal: destination path 'IR_project' already exists and is not an empty directory.
/content/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project


In [166]:
queries_path = 'queries.tsv'
df = pd.read_csv(queries_path, sep='\t',header = None, names=['ID', 'Text'])
queries = df['Text'].tolist()
queries = [query.upper() for query in queries]

print(f'Loaded {len(queries)} queries and {len(collection)} passages')

Loaded 20 queries and 1209 passages


In [169]:
# INDEXING
nbits = 2
doc_maxlen = 300
max_id = 10000
# Load pre-trained colBERT Model
checkpoint = '/content/drive/MyDrive/IR/colbertv2.0'
#checkpoint = 'colbert-ir/colbertv2.0'
index_name = f'cystic_f.{nbits}bits'

print("Index Name :", index_name)

Index Name : cystic_f.2bits


In [172]:
# Index run on 100 first docs of collection - Estimated time : 5m 30s

In [171]:
with Run().context(RunConfig(nranks=1, experiment='notebook')):
  config = ColBERTConfig(doc_maxlen=doc_maxlen, nbits=nbits, kmeans_niters=4)
  indexer = Indexer(checkpoint=checkpoint, config=config)
  try:
    indexer.index(name=index_name, collection=collection[:100], overwrite=True)
    indexer.get_index()
  except Exception as e:
    print(f"An error occurred: {e}")
  print("Everything OK - Creation of Index Complete")



[Jan 30, 15:26:06] #> Note: Output directory /content/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/experiments/notebook/indexes/cystic_f.2bits already exists


[Jan 30, 15:26:06] #> Will delete 10 files already at /content/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/experiments/notebook/indexes/cystic_f.2bits in 20 seconds...
#> Starting...
#> Joined...
Everything OK - Creation of Index Complete


In [173]:
# SEARCHING
with Run().context(RunConfig(experiment='notebook')):
  searcher= Searcher(index=index_name, collection=collection)


[Jan 30, 15:34:53] #> Loading codec...
[Jan 30, 15:34:53] #> Loading IVF...
[Jan 30, 15:34:53] #> Loading doclens...


/usr/local/lib/python3.10/dist-packages/torch/cuda/amp/grad_scaler.py:125: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
100%|██████████| 1/1 [00:00<00:00, 990.86it/s]

[Jan 30, 15:34:53] #> Loading codes and residuals...



100%|██████████| 1/1 [00:00<00:00, 327.25it/s]


In [240]:
# Find k top passages for this query - COLBERT MODEL
def colbert_find_top_k(query, k):
  print(f"#> {query}")
  print(f"Top {k} passages:")
  print("\t Rank \t\t Score \t\t Passage_ID ")
  results = searcher.search(query, k)

  # Print out the top-k retrieved passages
  for passage_id, passage_rank, passage_score in zip (*results):
    print(f"\t [{passage_rank}] \t\t {passage_score:.1f} \t\t {searcher.collection[passage_id]}")


In [243]:
col_dict = {}
col_scaled = {}
for i, query in enumerate(queries, start=1):
  results = searcher.search(query, k=10)
  query_id = f"{i:05d}"

  # Extract top-K relevance Scores for each passage
  passage_scores = {passage_id: passage_score for passage_id, _, passage_score in zip(*results)}
  relevances = list(passage_scores.values())
  min_relevance = np.min(relevances)
  max_relevance = np.max(relevances)

  # Scale the relevance scores and store in the scaled_dict with scale (0,1)
  scaled_relevances = [(passage_id, (relevance - min_relevance) / (max_relevance - min_relevance))
                        for passage_id, relevance in zip(passage_scores.keys(), relevances)]

  col_scaled[query_id] = dict(scaled_relevances)
  col_dict[query_id] = passage_scores

print("\nRelevance Scores for each Query - Sorted")
print(col_dict)
print("\nRelevant Scores for each Query - Sorted - Scaled (colBERT)")
print(col_scaled)



Relevance Scores for each Query - Sorted
{'00001': {64: 15.944551467895508, 15: 15.11480712890625, 99: 14.586509704589844, 29: 14.377805709838867, 87: 14.21850872039795, 89: 13.820152282714844, 13: 13.717822074890137, 48: 13.521918296813965, 83: 13.045907020568848, 84: 12.995705604553223}, '00002': {64: 15.844244003295898, 91: 15.369537353515625, 15: 14.794914245605469, 87: 14.59117603302002, 75: 14.485010147094727, 56: 14.10970401763916, 29: 14.033432960510254, 83: 13.886457443237305, 13: 13.674847602844238, 65: 13.556596755981445}, '00003': {29: 17.639225006103516, 71: 17.409854888916016, 94: 17.26728630065918, 77: 16.682157516479492, 83: 16.666120529174805, 74: 15.45413589477539, 40: 15.238809585571289, 8: 15.087007522583008, 15: 14.832198143005371, 87: 14.747379302978516}, '00004': {64: 15.12001895904541, 65: 15.095756530761719, 75: 14.39460563659668, 89: 14.180282592773438, 94: 14.07453727722168, 79: 13.831984519958496, 16: 13.608294486999512, 90: 13.391072273254395, 71: 13.37292

In [262]:
# Dictinoary of top-K passages for specific query
queries_path = 'queries_free.tsv'
queries = Queries(queries_path)
rankings = searcher.search_all(queries, k=5).todict()
print("\nDictionary of Relevance Scores - Ranked")
print(rankings)

def dict_to_list(entry):
    doc_id = entry[0]
    rank = entry[1]
    score = entry[2]
    return (doc_id, rank, score)

# List for relevance scores - Ranked (colBERT)
rankings_col = [[dict_to_list(entry) for entry in entries] for entries in rankings.values()]

print("\nRelevance scores for all Queries - Ranked (colBERT)")
print(rankings_col)

[Jan 30, 17:53:33] #> Loading the queries from queries_free.tsv ...
[Jan 30, 17:53:33] #> Got 19 queries. All QIDs are unique.



/usr/local/lib/python3.10/dist-packages/torch/amp/autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
19it [00:00, 55.53it/s]


Dictionary of Relevance Scores - Ranked
{1: [(64, 1, 15.944551467895508), (15, 2, 15.11480712890625), (99, 3, 14.586508750915527), (29, 4, 14.377805709838867), (87, 5, 14.21850872039795)], 2: [(64, 1, 15.844243049621582), (91, 2, 15.369537353515625), (15, 3, 14.794913291931152), (87, 4, 14.591175079345703), (75, 5, 14.485010147094727)], 3: [(29, 1, 17.639225006103516), (71, 2, 17.409852981567383), (94, 3, 17.267288208007812), (77, 4, 16.682157516479492), (83, 5, 16.666118621826172)], 4: [(64, 1, 15.120019912719727), (65, 2, 15.095755577087402), (75, 3, 14.394604682922363), (89, 4, 14.180282592773438), (94, 5, 14.074535369873047)], 5: [(75, 1, 20.112125396728516), (64, 2, 19.69268226623535), (86, 3, 19.651880264282227), (87, 4, 18.089618682861328), (90, 5, 17.973682403564453)], 6: [(79, 1, 11.795584678649902), (15, 2, 11.628297805786133), (32, 3, 11.517963409423828), (41, 4, 11.277161598205566), (81, 5, 11.236893653869629)], 7: [(32, 1, 16.737747192382812), (71, 2, 16.186765670776367),

In [248]:
# Dependencies for processing the theoritical document relevance
!pip install nltk
!git clone https://github.com/AurumAsinus/IR_project.git
%cd IR_project
!git pull origin main
!git fetch --all
!git reset --hard origin/main
import importlib
# File Handling given data from my GitHub Repo
import scores
importlib.reload(scores)

fatal: destination path 'IR_project' already exists and is not an empty directory.
/content/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 9 (delta 5), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (9/9), 3.53 KiB | 723.00 KiB/s, done.
From https://github.com/AurumAsinus/IR_project
 * branch            main       -> FETCH_HEAD
   fcd1792..56563ec  main       -> origin/main
Updating fcd1792..56563ec
Fast-forward
 Step2.py         |  5 +++--
 queries_free.tsv | 19 +++++++++++++++++++
 scores.py        |  2 +-
 3 files changed, 23 insertions(+), 3 deletions(-)
 create mode 100644 queries_free.tsv
Fe

<module 'scores' from '/content/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/IR_project/scores.py'>

In [251]:
# IDEAL RELEVANT DOCUMENTS (given values)
# --------------------------------------------
from scores import read_query_file, keep_top_5, extract_relevance_values, scale_relevance_values, scale_dictionary

# List of Relevance Scores for top-K Ranked
rankings_th = []
file_path = "cfquery_detailed"
query_data = read_query_file(file_path)
rankings_th = keep_top_5(query_data)


# List of Relevance Scores for all docs
sorted_th = []
for query_id, data in query_data.items():
        scores = data['scores']
        res = sorted(scores.items(), key=lambda x: x[1], reverse=True)
        sorted_th.append([(doc_id, rank + 1, score) for rank, (doc_id, score) in enumerate(res)])

# Dictionary of Relevance Scores for top-K Ranked
rankings_dict = {}
for i, query_rankings in enumerate(rankings_th, start=1):
  rankings_dict[i] = query_rankings

print("\nDictionary of Relevance Scores for top-K Ranked")
print(rankings_dict)

# Dictionary of Relevance Score for all docs
sorted_dict = {}
for i, query_rankings in enumerate(sorted_th, start=1):
  sorted_dict[i] = query_rankings

print("\nDictionary of Relevance Scores for all docs")
print(sorted_dict)

# Print Rankings
# for i, query_rankings in enumerate(rankings, start=1):
#     print(f"\nQuery {i} Rankings:")
#     for doc_id, rank, score in query_rankings:
#         print(f"Doc ID: {doc_id}, Rank: {rank}, Score: {score}")

# Extracting relevance values
relevance_values = extract_relevance_values(query_data)

# Scaled relevance values
scaled_relevance = scale_relevance_values(relevance_values)

# Scaled dictionary with relevance scores in (0,1)
scaled_dict = scale_dictionary(query_data, relevance_values)

print("\nRelevance Scores for each Query - Scaled")
print(scaled_dict)


Dictionary of Relevance Scores for top-K Ranked
{1: [(533, 1, 2222), (151, 2, 2211.0), (441, 3, 2122), (139, 4, 1222.0), (461, 5, 1121)], 2: [(875, 1, 1011.0), (434, 2, 1001.0), (169, 3, 1000.0), (498, 4, 1000.0), (499, 5, 1000.0)], 3: [(633, 1, 2222), (375, 2, 2221), (139, 3, 2122.0), (856, 4, 2112), (454, 5, 1220)], 4: [(604, 1, 2222.0), (876, 2, 2222), (711, 3, 2122), (669, 4, 1010.0), (538, 5, 100.0)], 5: [(151, 1, 2222), (374, 2, 2222), (439, 3, 2222), (593, 4, 2222), (875, 5, 2222)], 6: [(31, 1, 2222.0), (503, 2, 1220), (875, 3, 1120), (592, 4, 1010), (975, 5, 1010)], 7: [(371, 1, 2122.0), (623, 2, 2121), (856, 3, 2111), (1064, 4, 1120), (1054, 5, 1100)], 8: [(683, 1, 2220), (1125, 2, 2220), (23, 3, 2112.0), (499, 4, 1122), (439, 5, 1100.0)], 9: [(414, 1, 2222.0), (794, 2, 2110.0), (165, 3, 1122.0), (992, 4, 1010), (1115, 5, 102)], 10: [(157, 1, 2222.0), (413, 2, 2222), (581, 3, 2222), (676, 4, 2122), (878, 5, 2122)], 11: [(796, 1, 2222), (909, 2, 2222), (991, 3, 2222), (798, 4,

In [252]:
# VSM MODEL
# -----------------------------------
# Load needed depnedencies for constructing VSM (STEP 2)
import importlib
import Step2
importlib.reload(Step2)
import numpy as  np
import math
from nltk.corpus import stopwords
import nltk
import pandas as pd
import os
from nltk.stem import PorterStemmer
nltk.download('stopwords')
docs_folder_path = '/content/drive/MyDrive/docs'
# Define stop words
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Mounted at /content/drive


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [268]:
# Constructing Vector Space Model
# Load queries from TSV file
query_file_path = "queries.tsv"
queries = pd.read_csv(query_file_path, sep='\t')
queries = list(queries['Query'])

# Import needed functions from my VSM module
from Step2 import load_docs, tfij, idfi, tfidf, const_doc_vectors, const_query_vectors, cosine_similarity

# Load documents
documents = load_docs(docs_folder_path)

# Create Inverse Dictionary
def create_term_occurences_dict(tfij_vectors, documents):
  term_occurences_dict = {}
  for tfij_vector in tfij_vectors:
    for term in tfij_vector:
      term_occurences_dict[term] = term_occurences_dict.get(term, 0) + 1
  return term_occurences_dict

# Construct TFij Vectors
tfij_vectors = tfij(documents)

# Construct IDFi Vector
idfi_vector = idfi(tfij_vectors)

# Construct TF-IDF Vectors
tfidf_vectors = tfidf(tfij_vectors, idfi_vector)

# Construct Document Vectors
doc_vectors = const_doc_vectors(tfidf_vectors)

# Construct Query Vectors
query_vectors = []
result_dict = {}

# Construct Query Vectors
max_vector_length = max(len(tfidf_vector) for tfidf_vector in tfidf_vectors)

# Construct Rankings List for VSM Model
rankings_vsm = [[] for _ in range(len(queries))]

for i, query in enumerate(queries, start=1):
    query_vector = const_query_vectors(query, idfi_vector, max_vector_length)
    #print(f"\nQuery Vector {i}: {query_vector}")
    similarities = []
    # Similarity
    for doc_id, doc_vector in zip(documents.keys(), doc_vectors):
        similarity = cosine_similarity(doc_vector, query_vector)
        if np.isnan(similarity) or similarity == 0.0:
            continue
        elif (similarity > 0.01):
            similarities.append((doc_id, similarity))

    # Sort docs for each query by relevance score
    similarities.sort(key=lambda x: x[1], reverse=True)
    # Select top-k ranked docs
    top_5 = similarities[:5]
    # Place relevance scores for each query in dictionary
    result_dict[f"{i:05d}"] = {doc_id: score for doc_id, score in similarities}

    for rank, (doc_id, score) in enumerate(top_5, start=1):
      rankings_vsm[i-1].append((doc_id, rank, score))

# Construct Dictinoary from List for Relevance Scores - Ranked
rankings_dict_new = {}
for i, query_rankings in enumerate(rankings_vsm, start=1):
  rankings_dict_new[i] = [(int(doc_id), rank, score) for rank, (doc_id, _, score) in enumerate(query_rankings, start = 1)]


print("\nRelevance score of each document for each query - Not Ranked - All queries")
print(result_dict)

print("\n Dictionary with relevance scores for each query - Ranked")
print(rankings_dict_new)




Relevance score of each document for each query - Not Ranked - All queries
{'00001': {'00532': 0.07734793566341712, '00559': 0.056270863799242, '00660': 0.05001604707540956, '01120': 0.044408320838520414, '01000': 0.041058542556467445, '00333': 0.039492929809577144, '01197': 0.03901616973100081, '00956': 0.02841435684947172, '00749': 0.016603535909738548, '00676': 0.014661148525604973}, '00002': {'01075': 0.23493460454677192, '00614': 0.22085930107025414, '00070': 0.20628201247484995, '00569': 0.20038214857291212, '00543': 0.19841136413476215, '00525': 0.19229158266519508, '01117': 0.1812488206527185, '00066': 0.17845597918582437, '00528': 0.1713428345555697, '00550': 0.1699638648773986, '01014': 0.16987969879660747, '00536': 0.16901578650879437, '01130': 0.16740478694340136, '00328': 0.15976389500731963, '01079': 0.15855762971592913, '00333': 0.15821129065738346, '00505': 0.15766303452594252, '00844': 0.1538344323871833, '00432': 0.1484571947900689, '01222': 0.14612900864911574, '006

In [256]:
# Dictionary of queries - relevant passages (IDEAL)
# Based on given data

for outer_key, inner_dict in query_data.items():
  inner_keys = [int(inner_key) for inner_key in inner_dict['scores'].keys()]
  ground_truth.append(inner_keys)

# Construct ground truth dictionary
ground_truth_dict = {i+1: relevance_list for i, relevance_list in enumerate(ground_truth)}

print("Relevant Passages for each Query - All Queries - Unsorted")
print("Ground Truth Dictinory")
print(ground_truth_dict)

Relevant Passages for each Query - All Queries - Unsorted
Ground Truth Dictinory
{1: [139, 151, 166, 311, 370, 392, 439, 440, 441, 454, 461, 502, 503, 505, 520, 522, 526, 527, 533, 593, 619, 737, 742, 789, 827, 835, 861, 875, 891, 921, 922, 1175, 1185, 1222], 2: [169, 434, 454, 498, 499, 592, 875], 3: [23, 40, 139, 190, 221, 246, 309, 311, 325, 345, 347, 356, 370, 374, 375, 439, 440, 454, 515, 520, 524, 526, 527, 533, 535, 560, 561, 571, 584, 604, 623, 633, 733, 742, 854, 856, 950, 967, 1144, 1161, 1172, 1175, 1196], 4: [503, 538, 539, 540, 553, 604, 669, 711, 876], 5: [23, 47, 50, 60, 114, 132, 135, 139, 151, 169, 189, 190, 197, 200, 226, 256, 265, 297, 298, 299, 311, 312, 325, 333, 343, 347, 349, 369, 370, 371, 374, 386, 392, 410, 420, 427, 428, 430, 434, 437, 439, 440, 441, 450, 461, 465, 478, 496, 497, 498, 499, 500, 501, 502, 503, 505, 511, 513, 516, 520, 524, 531, 533, 549, 553, 559, 561, 568, 590, 592, 593, 604, 605, 633, 669, 701, 702, 710, 711, 722, 724, 729, 731, 733, 750, 76

In [258]:
print("Scores from each model - All Queries - Unsorted - Scaled in a scale in (0,1)")

# Scores from Ideal Data
print("\n\nScores from given Ideal Data: (top-K)")
print(scaled_dict)
# Scores from VSM Model
print("\n\nScores from VSM Model:")
print(result_dict)
# Scores from ColBERT
print("\n\nScores from ColBERT Model:  (top-K)")
print(col_scaled)

Scores from each model - All Queries - Unsorted - Scaled in a scale in (0,1)


Scores from given Theoritical Data: (top-K)
{'00001': {'scores': {139: 0.5497523638000901, 151: 0.995047276001801, 166: 0.0, 311: 0.0, 370: 0.4542998649257091, 392: 0.0, 439: 0.0, 440: 0.004502476361999099, 441: 0.954975236380009, 454: 0.044574515983791085, 461: 0.5042773525438992, 502: 0.00045024763619990995, 503: 0.44979738856371004, 505: 0.0, 520: 0.4542998649257091, 522: 0.0, 526: 0.45475011256190906, 527: 0.45024763619990993, 533: 1.0, 593: 0.44979738856371004, 619: 0.0, 737: 0.044574515983791085, 742: 0.044574515983791085, 789: 0.0, 827: 0.00405222872579919, 835: 0.0, 861: 0.0, 875: 0.5042773525438992, 891: 0.0, 921: 0.4542998649257091, 922: 0.4542998649257091, 1175: 0.044574515983791085, 1185: 0.0, 1222: 0.0}}, '00002': {'scores': {169: 0.44979738856371004, 434: 0.45024763619990993, 454: 0.044574515983791085, 498: 0.44979738856371004, 499: 0.44979738856371004, 592: 0.00045024763619990995, 875: 0.45475

In [263]:

def show_rankings(rankings):
  for i, query_rankings in enumerate(rankings, start=1):
      print(f"\nQuery {i} Rankings:")
      for doc_id, rank, score in query_rankings:
          print(f"Doc ID: {doc_id}, Rank: {rank}, Score: {score}")


def show_top3_rankings_for_query(rankings, query_number):
    if 1 <= query_number <= len(rankings):
        query_rankings = rankings[query_number - 1]
        print(f"\nTop 3 Rankings for Query {query_number}:")
        for doc_id, rank, score in query_rankings[:3]:
            print("Rank:", rank)
            print(collection[int(doc_id)])
    else:
        print(f"Invalid query number. Please provide a valid query number.")

print("\n\nShow Ranknings from Theoritical Model:")
#show_rankings(rankings_th)
print("\n\nShow Ranknings from VSM Model:")
#show_rankings(rankings_vsm)
print("\n\nShow Ranknings from colBERT Model:")
#show_rankings(rankings_col)



Show Ranknings from Theoritical Model:


Show Ranknings from VSM Model:


Show Ranknings from colBERT Model:


In [264]:

# Create a new ground truth dictionary
ground_truth_dict = {}

for query_id, ranked_docs in sorted_dict.items():
    # Extract relevant document IDs from the ranking, preserving the order
    relevant_docs = [doc_id for doc_id, _, _ in ranked_docs]

    # Update the ground truth for the current query
    ground_truth_dict[query_id] = relevant_docs

# Print the ground truth dictionary
print(ground_truth_dict)



{1: [533, 151, 441, 139, 461, 875, 526, 370, 520, 921, 922, 527, 503, 593, 454, 737, 742, 1175, 440, 827, 502, 166, 311, 392, 439, 505, 522, 619, 789, 835, 861, 891, 1185, 1222], 2: [875, 434, 169, 498, 499, 454, 592], 3: [633, 375, 139, 856, 454, 527, 440, 1144, 526, 742, 370, 23, 309, 515, 733, 1172, 854, 311, 40, 345, 347, 356, 520, 524, 535, 560, 571, 584, 623, 950, 604, 1196, 190, 221, 246, 325, 374, 439, 533, 561, 967, 1161, 1175], 4: [604, 876, 711, 669, 538, 539, 540, 503, 553], 5: [151, 374, 439, 593, 875, 47, 503, 23, 1064, 710, 256, 533, 805, 895, 499, 633, 497, 856, 461, 496, 980, 975, 1144, 861, 370, 498, 1175, 511, 500, 325, 516, 524, 465, 386, 430, 553, 592, 520, 702, 857, 1196, 867, 501, 502, 605, 750, 772, 347, 114, 139, 265, 440, 513, 561, 669, 343, 604, 711, 724, 731, 733, 843, 876, 888, 889, 982, 990, 1000, 1092, 1093, 50, 60, 132, 135, 169, 189, 190, 197, 200, 226, 297, 298, 299, 311, 312, 333, 349, 369, 371, 392, 410, 420, 427, 428, 434, 437, 441, 450, 478, 505, 5

In [265]:

# recall@K
def recall_k(rankings, ground_truth, k):
  total_queries = len(rankings)
  #print("Total queries: ", total_queries)
  total_recall_at_k = 0.0

  for query_id, retrieved_passages in rankings.items():
    relevant_passages = ground_truth.get(query_id, [])
    #print(relevant_passages)
    retrieved_at_k = [passage_id for passage_id, _, _ in retrieved_passages[:k]]
    #print(retrieved_at_k)
    # Calculate recall@k for current query
    recall_at_k = len(set(retrieved_at_k).intersection(relevant_passages)) / len(relevant_passages)
    print(recall_at_k)
    total_recall_at_k += recall_at_k
  # Average recall over all queries
  avg_recall_at_k = total_recall_at_k/ total_queries
  print("Recall@K = ", round(avg_recall_at_k, 2))
  return avg_recall_at_k


In [266]:
# Mean Reciprocal Rank
def MRR(rankings, k):
  Q = len(rankings)
  cumulative_reciprocal = 0
  for query_id, retrieved_passages in rankings.items():
    first_result = [passage_id for passage_id, _, _ in retrieved_passages[:k]]
    first_result = int(first_result[0])
    reciprocal = 1 / first_result
    cumulative_reciprocal += reciprocal
    print(f"query #{query_id} = 1/{first_result} = {reciprocal}")
  mrr = (1/Q) * cumulative_reciprocal
  print("MRR = ", round(mrr, 2))


In [213]:
# Mean Average Precision
def MAP(rankings, ground_truth, k):
  Q = len(rankings)
  ap = []
  for query_id, retrieved_passages in rankings.items():
    ap_num = 0
    # loop through k values
    act = ground_truth.get(query_id, [])
    act_set = set(act)
    x = 0
    pred = []
    for passage_id, _, _ in retrieved_passages[:k]:
      pred.append(passage_id)
      pred_set = set(pred)
      precision_at_k = len(act_set & pred_set) / (x+1)
      # calcualte rel_k values
      if pred[x] in act:
        rel_k = 1
      else :
        rel_k = 0
      ap_num += precision_at_k * rel_k
      x+=1
    ap_q = ap_num / len(act)
    print(f"AP@{k}_{query_id} = {round(ap_q, 2)}")
    ap.append(ap_q)
  map_at_k = sum(ap) / Q

  print(f"mAP@{k} = {round(map_at_k, 2)}")

In [72]:
# from math import log2
# # Normalized Discounted Cumulative Gain
# def NDCG(rankings, k):
#   dcg = 0
#   for query_id, retrieved_passages in rankings.items():
#     for passage_id, rank, _ in retrieved_passages[:k]:
#       rel_k = k-rank
#       dcg += rel_k/log2(1+k)

# NDCG(rankings, 5)



In [269]:
# RUN THIS CELL TO CHOOSE WHAT FUNCTIONALITY YOU WANT TO TRY
# !!! All the previous cells should have been run !!!

def choice_1():
  print("\nInverse Dictionary of Terms' Occurences\n")
  inverse_dict = create_term_occurences_dict(tfij_vectors, documents)
  print(inverse_dict)

def choice_2():
  print("Vector Space Model Implementation")
  choice = int(input())
  print("You can choose among:")
  print("(1) : See top Rankings for all queries")
  print("(2) : See Rank of Docs for a Query")
  if (choice == 1):
    show_rankings(rankings_vsm)
  elif(choice == 2):
    print("Give Query Number (0-19)")
    query = int(input())
    query_id = f"{query:05d}"
    if query_id in rankings_dict_new:
      scores_for_query = rankings_dict_new[query_id]
      doc_score_pairs = [(doc_id, score) for doc_id, score in scores_for_query.items()]
      print(f"Pairs of doc_id, score for query '{query_id}': {doc_score_pairs}")
  else:
    print("Invalid. Try Again...")
    choice = int(input())

def choice_3():
  print("colBERT Model Implementation")
  choice = int(input())
  print("You can choose among:")
  print("(1) : Print out top-k passages for a query ")
  print("(2) : Show Rankings for all queries")
  if (choice == 1):
    print("Select query number (0-19)")
    choice = int(input())
    print("Select # of most releveant docs to appear")
    k = int(input())
    colbert_find_top_k(queries[choice], k)
  elif(choice == 2):
    show_rankings(rankings_col)
  else:
    print("Invalid. Try Again...")
    choice = int(input())

def choice_4():
  print("Compare Models - Metrics Implemetations")
  print("Select Model to be see metrics:\n(1) : Ideal\n(2) : VSM\n(3) : ColBERT")
  choice_model = int(input())
  print("Select Metric to be used\n(1) : Recall@K\n(2) : MRR\n(3) : MAP")
  choice_metric = int(input())
  if (choice_model == 1):
    if (choice_metric == 1):
      print("Recall for Ideal: ")
      recall_at_k = recall_k(rankings_dict, ground_truth_dict, k=5)
    elif (choice_metric == 2):
      print("MRR for Ideal: ")
      MRR(rankings_dict, 5)
    elif (choice_metric == 3):
      print("MAP for Ideal: ")
      MAP(rankings_dict, ground_truth_dict, 5)
  elif(choice_model == 2):
    if (choice_metric == 1):
      print("Recall for VSM: ")
      recall_at_k = recall_k(rankings_dict_new, ground_truth_dict, k=5)
    elif (choice_metric == 2):
      print("MRR for VSM: ")
      MRR(rankings_dict_new, 5)
    elif (choice_metric == 3):
      print("MAP for VSM: ")
      MAP(rankings_dict_new, ground_truth_dict, 5)
  elif(choice_model == 3):
    if (choice_metric == 1):
      print("Recall for colBERT: ")
      recall_at_k = recall_k(rankings, ground_truth_dict, k=5)
    elif (choice_metric == 2):
      print("MRR for colBERT: ")
      MRR(rankings, 5)
    elif (choice_metric == 3):
      print("MAP for colBERT: ")
      MAP(rankings, ground_truth_dict, 5)
  else:
    print("Invalid Model Choice. Try Again...")
    choice = int(input())

def choice_5():
  print("Select Model to be used\n(1) : Ideal\n(2) : VSM\n(3) : ColBERT")
  choice = int(input())
  print("Select No of Query (Test from 1-20)")
  query_number = int(input())
  print("Print 3 Most Relevant Docs")
  if (choice == 1):
    show_top3_rankings_for_query(rankings_th, query_number)
  elif(choice == 2):
    show_top3_rankings_for_query(rankings_vsm, query_number)
  elif(choice == 3):
    show_top3_rankings_for_query(rankings_col, query_number)
  else:
    print("Invalid. Try Again...")
    choice = int(input())


print("SELECT FUNCTIONALITY: (1-4)")
print("(1) STEP 1: Inspect Inverse Dictionary ")
print("(2) STEP 2: Inspect Vector Space Model")
print("(3) STEP 3: Inspect colBERT Model")
print("(4) STEP 4: Check Metrics")
print("(5) STEP 5: Inspect Models by Using Search Engine")

switch_dict = {
    1: choice_1,
    2: choice_2,
    3: choice_3,
    4: choice_4,
    5: choice_5
}
user_choice = int(input("Enter your choice: "))

def invalid_choice():
  print("Invalid. Please enter a number between 0-4")

switch_dict.get(user_choice, invalid_choice())()




SELECT FUNCTIONALITY: (1-4)
(1) STEP 1: Inspect Inverse Dictionary 
(2) STEP 2: Inspect Vector Space Model
(3) STEP 3: Inspect colBERT Model
(4) STEP 4: Check Metrics
(5) STEP 5: Inspect Models by Using Search Engine
Enter your choice: 2
Invalid. Please enter a number between 0-4
Vector Space Model Implementation
2
You can choose among:
(1) : See top Rankings for all queries
(2) : See Rank of Docs for a Query
Give Query Number (0-19)
2


In [ ]:
1
